In [1]:
import json
import os
import numpy as np

In [2]:
os.chdir('/Users/supperxxxs/Desktop/kaggle/cooking')
train = json.loads(open('train.json').read())
test = json.loads(open('test.json').read())

In [3]:
cat = set()
lbl = set()
sample = set()

for k in train:
    ing = k['ingredients']
    for u in ing:
        cat.add(u)
    lbl.add(k['cuisine'])
    sample.add(k['id'])

num_of_ingredients = len(cat)
num_of_cuisine = len(lbl)
num_of_sample = len(sample)
print(num_of_cuisine)
print(num_of_ingredients)
print(len(sample))

cat_map = {}
for i,t in enumerate(list(cat)):
    cat_map[t] = i
#print(cat_map)
lbl_map = {}
for i,t in enumerate(list(lbl)):
    lbl_map[t] = i
print (lbl_map)

20
6714
39774
{'chinese': 0, 'british': 1, 'thai': 2, 'japanese': 3, 'mexican': 4, 'korean': 5, 'indian': 6, 'greek': 7, 'cajun_creole': 8, 'southern_us': 9, 'filipino': 10, 'jamaican': 11, 'brazilian': 12, 'irish': 13, 'spanish': 14, 'french': 15, 'vietnamese': 16, 'russian': 17, 'italian': 18, 'moroccan': 19}


In [4]:
train_data = []
train_label = []
features  = []
for k in train:
    ing = k['ingredients']
    features = [0]*num_of_ingredients
    for u in ing:
        features[cat_map[u]] = 1
    train_data.append(features)
    train_label.append(lbl_map[k['cuisine']])


train_data, train_label = np.array(train_data), np.array(train_label)
train_data.shape, train_label.shape
print(train_label)
print(train_data)

[ 7  9 10 ... 13  0  4]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [5]:
print(test[0]['ingredients'])

['baking powder', 'eggs', 'all-purpose flour', 'raisins', 'milk', 'white sugar']


In [6]:
test_data = []
cat1 = set()

for a in test:
    ing = a['ingredients']
    for b in ing:
        cat1.add(b) 
number_of_ingredient = len(cat) 
    
for a in test:
    ing = a['ingredients'] 
    features = [0]*number_of_ingredient # use the same length of the list in train data set
    for b in ing:
        if b not in cat_map.keys():
            continue
        features[cat_map[b]] = 1
    test_data.append(features)
test_data = np.array(test_data)

In [7]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_data, train_label)
from sklearn import metrics, cross_validation
pred = cross_validation.cross_val_predict(logreg, train_data, train_label,cv=5)
accuracy = metrics.accuracy_score(train_label,pred)
print(accuracy)


/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.7799567556695328


In [8]:
logreg.fit(train_data, train_label)
result = logreg.predict(test_data)

In [9]:
import pandas as pd
test = pd.read_json('test.json')

inv_map = {}
for i in lbl_map:
    inv_map[lbl_map[i]]=i
temp = []
for i in result:
    temp.append(inv_map[i]);

test['cuisine'] = temp
test = test.drop('ingredients', axis = 1)
test.to_csv('output.csv',index = False)
